In [36]:
import requests
from urllib.parse import urlparse
from readability import Document
from bs4 import BeautifulSoup

In [3]:
url = 'https://admissions.utah.edu/information-resources/residency/residency-state-exceptions/'

In [5]:
URL = urlparse(url)

In [6]:
page_content = requests.get(url).text

In [9]:
from IPython.display import HTML, display

# Display The HTML

In [11]:
display(HTML("<iframe>"+page_content+"</iframe>"))

# Document.content()

In [24]:
page_content_doc = Document(page_content)
display(HTML(
    "<iframe>"+
    page_content_doc.content().strip("'b")
    .replace('\\t', '\t')
    .replace('\\n', '\n')+"</iframe>"))

# Document.get_clean_html()

In [40]:
BeautifulSoup(
    page_content_doc.get_clean_html(),
    features="lxml"
)#.pretty()

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<noscript><img src="https://www.facebook.com/tr?id=2834829919982746&amp;ev=PageView&amp;noscript=1"/></noscript>
<meta content="The University of Utah, UofU, The U, Residency – State Exceptions - Utah Admissions" name="keywords"/>
<meta content="Utah Admissions" name="description"/>
<meta content="Utah Admissions" property="og:description"/>
<meta content="Residency – State Exceptions - Utah Admissions" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://admissions.utah.edu/information-resources/residency/residency-state-exceptions/" property="og:url"/>
<meta content="https://admissions.utah.edu/wp-content/uploads/sites/6/2022/07/BackToSchool-Oktoberfest-26_2000.jpg" property="og:image"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="Residency – State Exceptions - Utah Admissions" name="twitter:title"/>

# BeautifulSoup.get_text()

In [55]:
import re

In [69]:
page_content_dom = BeautifulSoup(page_content)

text = re.sub(r'\n\n\n+', '\n', page_content_dom.get_text().strip('\n'))
text

'Residency - State Exceptions | Admissions - The University of Utah\nMain MenuApplyToggle Sub Menu\n\nApply\nTrack Your Application\nFreshman StudentsToggle Sub Menu\n\nFreshman Students\nUndergraduate Admissions Standards\nTransfer StudentsToggle Sub Menu\n\nTransfer Students\nTransfer Admission Standards\nGraduate Admissions\nInternationalToggle Sub Menu\n\nInternational\nInternational Freshman Admissions & Undergraduate Costs\nInternational Graduate Admissions & Graduate Costs\nInternational Transfer Admissions\nInternational Non-Traditional\nInternational Returning Admissions\nInternational Second Bachelor’s Admissions\nEnglish Proficiency\nI-20 Information\nReturning Students\nSecond Bachelor’s Admissions\nNon-Traditional\nNon-Degree Seeking\nHigh School University Program & Orientation\nEarly Admission\nRequest Info\nVisit & EngageToggle Sub Menu\n\nVisit & Engage\nVirtual Campus Tour\nFind Your Counselor\nMeet Our Student Ambassadors\nUpcoming Events\nAdmitted StudentsToggle Sub

# recursive_split

In [71]:
def recursive_split(text, max_length, split_characters, final_split_character):
    
    text = text.strip()
    if len(text)<max_length:
        return text

    if len(split_characters)==0:
        chunked = text.split(final_split_character)
        half_split_idx = len(chunked)//2
        first_half_chunk = final_split_character.join(chunked[:half_split_idx]).strip()
        second_half_chunk = final_split_character.join(chunked[half_split_idx:]).strip()
        if len(first_half_chunk)<=max_length:
            yield first_half_chunk
            return
        else:
            for chunk_i in recursive_split(first_half_chunk, max_length, [], final_split_character):
                yield chunk_i
            return
        if len(second_half_chunk)<=max_length:
            yield second_half_chunk
            return
        else:
            for chunk_i in recursive_split(second_half_chunk, max_length, [], final_split_character):
                yield chunk_i
            return
    
    for chunk in text.split(split_characters[0]):
        chunk = chunk.strip()
        if len(chunk)<=max_length:
            yield chunk
        else:
            for chunk_i in recursive_split(chunk, max_length, split_characters[1:], final_split_character):
                yield chunk_i

In [80]:
list(recursive_split(text, 500, ['\n\n', '\n'], '. '))

['Residency - State Exceptions | Admissions - The University of Utah\nMain MenuApplyToggle Sub Menu',
 'Apply\nTrack Your Application\nFreshman StudentsToggle Sub Menu',
 'Freshman Students\nUndergraduate Admissions Standards\nTransfer StudentsToggle Sub Menu',
 'Transfer Students\nTransfer Admission Standards\nGraduate Admissions\nInternationalToggle Sub Menu',
 'International\nInternational Freshman Admissions & Undergraduate Costs\nInternational Graduate Admissions & Graduate Costs\nInternational Transfer Admissions\nInternational Non-Traditional\nInternational Returning Admissions\nInternational Second Bachelor’s Admissions\nEnglish Proficiency\nI-20 Information\nReturning Students\nSecond Bachelor’s Admissions\nNon-Traditional\nNon-Degree Seeking\nHigh School University Program & Orientation\nEarly Admission\nRequest Info\nVisit & EngageToggle Sub Menu',
 'Visit & Engage\nVirtual Campus Tour\nFind Your Counselor\nMeet Our Student Ambassadors\nUpcoming Events\nAdmitted StudentsTogg

# overlap_split

In [84]:
def overlap_split(text, max_chunk_size, chunk_overlap):
    
    l = len(text)
    
    i = max_chunk_size
    yield text[:max_chunk_size]
    
    while True:
        yield text[ i - chunk_overlap : i + max_chunk_size + chunk_overlap ]
        if i + max_chunk_size + chunk_overlap > l:
            break
        i += max_chunk_size

In [89]:
list(overlap_split(text, 200, 100))[0]

'Residency - State Exceptions | Admissions - The University of Utah\nMain MenuApplyToggle Sub Menu\n\nApply\nTrack Your Application\nFreshman StudentsToggle Sub Menu\n\nFreshman Students\nUndergraduate Admissi'